# PDFドキュメント分析 - Snowflake Key Concepts

Snowflake Cortex AI関数を使用してPDFドキュメントを分析します。

**分析内容:**
1. AI_PARSE_DOCUMENT - ドキュメント解析・画像抽出
2. AI_EXTRACT - 機能名・キーワード抽出
3. AI_AGG - ドキュメント要約
4. AI_COMPLETE (Multimodal) - 抽出画像の分析
5. SPLIT_TEXT_MARKDOWN_HEADER - チャンク分割

**使用データ:** `@ai_discover.public.raw_stage/pdf/snowflake_key_concept.pdf`

In [6]:
%%sql -r dataframe_2
USE SCHEMA ai_discover.public;

---
## 1. AI_PARSE_DOCUMENT - ドキュメント解析・画像抽出

`AI_PARSE_DOCUMENT`はPDF、Word、画像からテキスト・レイアウト・画像を抽出するCortex AI関数です。

**特徴:**
- PDF, DOCX, PPTX, 画像ファイルに対応
- LAYOUTモードで表・ヘッダー構造を保持
- `extract_images: true`で埋め込み画像を抽出

**構文:**
```sql
AI_PARSE_DOCUMENT(
    TO_FILE('@stage/document.pdf'),
    {'mode': 'LAYOUT', 'extract_images': true}
)
```

In [7]:
%%sql -r parse_result
CREATE OR REPLACE TABLE DOC_PARSED AS
SELECT 
    'snowflake_key_concept.pdf' AS document_name,
    AI_PARSE_DOCUMENT(
        TO_FILE('@ai_discover.public.raw_stage/pdf/snowflake_key_concept.pdf'),
        {'mode': 'LAYOUT', 'extract_images': true, 'page_split': true}
    ) AS parsed_content

In [21]:
%%sql -r dataframe_1
SELECT PARSED_CONTENT:pages[0].content as content FROM DOC_PARSED LIMIT 100;

In [4]:
from snowflake.snowpark.context import get_active_session
import json
import re
session = get_active_session()
df = session.sql('SELECT PARSED_CONTENT FROM DOC_PARSED').to_pandas()
parsed = df['PARSED_CONTENT'].iloc[0]
if isinstance(parsed, str):
    parsed = json.loads(parsed)
page_count = len(parsed.get('pages', []))
total_images = sum(len(page.get('images', [])) for page in parsed.get('pages', []))
print(f'ページ数: {page_count}')
print(f'抽出画像数: {total_images}')

full_text = '\n\n'.join(page.get('content', '') for page in parsed.get('pages', []))
sections = re.split(r'(^#{1,2}\s+.+$)', full_text, flags=re.MULTILINE)
print('\n=== 章・節構成（先頭3件）===')
count = 0
for i, section in enumerate(sections):
    if re.match(r'^#{1,2}\s+', section):
        level = '章' if section.startswith('# ') else '節'
        title = section.strip()
        content = sections[i+1].strip()[:200] if i+1 < len(sections) else ''
        print(f'\n【{level}】{title}')
        print(f'{content}...')
        count += 1
        if count >= 3:
            break

In [6]:
%%sql -r save_images_proc
CREATE OR REPLACE PROCEDURE SAVE_EXTRACTED_IMAGES(r VARIANT)
RETURNS ARRAY
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
PACKAGES = ('pillow', 'snowflake-snowpark-python')
HANDLER = 'run'
AS
$$
import base64
import io
import os
import tempfile
from PIL import Image

def process_parse_document_result(data: dict) -> tuple[str, str, str]:
    for page in data.get("pages", []):
        for image in page.get("images", []):
            id = image["id"]
            data_str, image_base64 = image["image_base64"].split(";", 1)
            extension = data_str.split("/")[1]
            base64_data = image_base64.split(",")[1]
            yield id, extension, base64_data

def decode_base64(encoded_image: str) -> bytes:
    return base64.b64decode(encoded_image)

def run(session, r):
    destination_path = r["DESTINATION_PATH"]
    parse_document_result = r["PARSE_DOCUMENT_RESULT"]
    if not destination_path or not destination_path.startswith("@"):
        return ["Error: destination_path must start with @"]
    uploaded_files = []
    with tempfile.TemporaryDirectory() as temp_dir:
        for image_id, extension, encoded_image in process_parse_document_result(parse_document_result):
            image_bytes = decode_base64(encoded_image)
            image: Image = Image.open(io.BytesIO(image_bytes))
            image_path = os.path.join(temp_dir, image_id)
            image.save(image_path)
            session.file.put(image_path, destination_path, auto_compress=False, overwrite=True)
            uploaded_files.append(f"{destination_path}/{image_id}")
            os.remove(image_path)
    return uploaded_files
$$

In [8]:
%%sql -r save_images_result
CALL SAVE_EXTRACTED_IMAGES(
    (SELECT OBJECT_CONSTRUCT(*)
     FROM (SELECT
         '@ai_discover.public.images' AS destination_path,
         PARSED_CONTENT AS parse_document_result
     FROM DOC_PARSED) LIMIT 1)
)

In [9]:
import tempfile
import os
from PIL import Image
import matplotlib.pyplot as plt

df = session.sql("SELECT * FROM DIRECTORY('@ai_discover.public.images')").to_pandas()
print(f'保存された画像数: {len(df)}')
for idx, row in df.iterrows():
    print(f"  - {row['RELATIVE_PATH']}")

with tempfile.TemporaryDirectory() as temp_dir:
    session.file.get('@ai_discover.public.images/img-0.jpeg', temp_dir)
    img_path = os.path.join(temp_dir, 'img-0.jpeg')
    img = Image.open(img_path)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('img-0.jpeg')
    plt.show()

---
## 2. AI_EXTRACT - 機能名・キーワード抽出

`AI_EXTRACT`は非構造化データから特定の情報を抽出するCortex AI関数です。

**特徴:**
- 質問形式で抽出内容を指定
- 複数の項目を一度に抽出可能
- 画像・テキストの両方に対応

**構文:**
```sql
AI_EXTRACT(text, {'feature_names': 'List all feature names mentioned'})
```

In [10]:
%%sql -r extract_result
CREATE OR REPLACE TABLE DOC_EXTRACTED AS
WITH full_text AS (
    SELECT LISTAGG(p.value:content::STRING, '\n\n') AS doc_text
    FROM DOC_PARSED, LATERAL FLATTEN(input => PARSED_CONTENT:pages) p
)
SELECT 
    doc_text,
    AI_EXTRACT(
        doc_text,
        {
            'feature_names': 'ドキュメント内で言及されているSnowflakeの機能名や製品名をすべてリストアップしてください（例: Snowpipe, Streams, Tasks等）',
            'key_concepts': 'このドキュメントで説明されている主要な技術コンセプトをリストアップしてください',
            'use_cases': '記載されている主なユースケースやメリットは何ですか？'
        }
    ) AS extracted_info
FROM full_text

In [11]:
df = session.sql('SELECT EXTRACTED_INFO FROM DOC_EXTRACTED').to_pandas()
extracted = df['EXTRACTED_INFO'].iloc[0]
if isinstance(extracted, str):
    extracted = json.loads(extracted)
response = extracted.get('response', extracted)
print('=== 抽出結果 ===')
for key, value in response.items():
    print(f'\n【{key}】')
    if isinstance(value, list):
        for item in value:
            print(f'  - {item}')
    else:
        print(f'  {value}')

---
## 3. AI_AGG - ドキュメント要約

`AI_AGG`は複数行のテキストを集約・要約するCortex AI関数です。

**特徴:**
- GROUP BYと組み合わせて複数レコードを要約
- カスタムプロンプトで要約形式を指定
- 長文を指定した文字数に圧縮

**構文:**
```sql
AI_AGG(text_column, 'Summarize in 300 words') ... GROUP BY ...
```

In [12]:
%%sql -r agg_result
CREATE OR REPLACE TABLE DOC_SUMMARY AS
SELECT 
    AI_AGG(
        p.value:content::STRING, 
        'Summarize the key points of this Snowflake documentation in 300 words. 
         Focus on the main features, benefits, and technical concepts. 
         Require that output format is JSON.'
    ) AS summary
FROM DOC_PARSED, LATERAL FLATTEN(input => PARSED_CONTENT:pages) p
GROUP BY 1=1

In [13]:
df = session.sql('SELECT SUMMARY FROM DOC_SUMMARY').to_pandas()
print('=== ドキュメント要約 ===')
print(df['SUMMARY'].iloc[0])

---
## 4. AI_COMPLETE (Multimodal) - 抽出画像の分析

`AI_COMPLETE`はマルチモーダル対応モデルで画像を分析できます。

**特徴:**
- pixtral-large等のマルチモーダルモデルを使用
- ステージ上の画像ファイルを直接分析
- 画像内のテキスト・グラフ・図表を理解

**構文:**
```sql
AI_COMPLETE(
    'pixtral-large',
    'Describe this image',
    TO_FILE('@stage/image.jpeg')
)
```

In [14]:
%%sql -r image_analysis_result
CREATE OR REPLACE TABLE DOC_IMAGE_ANALYSIS AS
SELECT 
    RELATIVE_PATH AS image_file,
    AI_COMPLETE(
        'claude-4-sonnet',
        'この画像を日本語で簡潔に説明してください。図表の場合は、何を表しているかを説明してください。',
        TO_FILE('@ai_discover.public.images/' || RELATIVE_PATH)
    ) AS image_description
FROM DIRECTORY('@ai_discover.public.images')
WHERE RELATIVE_PATH LIKE '%.jpeg' OR RELATIVE_PATH LIKE '%.png'
LIMIT 5

In [15]:
df = session.sql('SELECT * FROM DOC_IMAGE_ANALYSIS').to_pandas()
print('=== 画像分析結果 ===')
for idx, row in df.iterrows():
    print(f"\n【{row['IMAGE_FILE']}】")
    desc = row['IMAGE_DESCRIPTION']
    if isinstance(desc, str):
        desc = json.loads(desc)
    if isinstance(desc, dict):
        response = desc.get('response', desc)
        if isinstance(response, dict):
            for key, value in response.items():
                print(f'  [{key}]')
                if isinstance(value, list):
                    for item in value:
                        print(f'    - {item}')
                else:
                    print(f'    {value}')
        else:
            print(f'  {response}')
    else:
        print(f'  {desc}')

---
## 5. SPLIT_TEXT_MARKDOWN_HEADER - チャンク分割

`SPLIT_TEXT_MARKDOWN_HEADER`はMarkdownヘッダーベースでテキストをチャンク分割するCortex AI関数です。

**特徴:**
- ヘッダー階層（#, ##, ###）を認識
- 各チャンクにヘッダー情報を付与
- チャンクサイズとオーバーラップを指定可能
- RAGパイプラインに最適

**構文:**
```sql
SPLIT_TEXT_MARKDOWN_HEADER(
    text,
    {'#': 'header_1', '##': 'header_2'},
    chunk_size,
    overlap
)
```

In [16]:
%%sql -r chunk_result
CREATE OR REPLACE TABLE DOC_CHUNKS AS
WITH full_text AS (
    SELECT LISTAGG(p.value:content::STRING, '\n\n') AS doc_text
    FROM DOC_PARSED, LATERAL FLATTEN(input => PARSED_CONTENT:pages) p
)
SELECT 
    c.index AS chunk_index,
    c.value:chunk::STRING AS chunk_text,
    c.value:headers::OBJECT AS chunk_headers,
    LENGTH(c.value:chunk::STRING) AS chunk_length
FROM full_text,
LATERAL FLATTEN(
    SNOWFLAKE.CORTEX.SPLIT_TEXT_MARKDOWN_HEADER(
        doc_text,
        OBJECT_CONSTRUCT('#', 'header_1', '##', 'header_2', '###', 'header_3'),
        1000,
        100
    )
) c

In [ ]:
from snowflake.snowpark.context import get_active_session
import json
import re
session = get_active_session()
df = session.sql('SELECT PARSED_CONTENT FROM DOC_PARSED').to_pandas()
parsed = df['PARSED_CONTENT'].iloc[0]
if isinstance(parsed, str):
    parsed = json.loads(parsed)
page_count = len(parsed.get('pages', []))
total_images = sum(len(page.get('images', [])) for page in parsed.get('pages', []))
print(f'ページ数: {page_count}')
print(f'抽出画像数: {total_images}')

full_text = '\n\n'.join(page.get('content', '') for page in parsed.get('pages', []))
sections = re.split(r'(^#{1,2}\s+.+$)', full_text, flags=re.MULTILINE)
print('\n=== 章・節構成（先頭3件）===')
count = 0
for i, section in enumerate(sections):
    if re.match(r'^#{1,2}\s+', section):
        level = '章' if section.startswith('# ') else '節'
        title = section.strip()
        content = sections[i+1].strip()[:200] if i+1 < len(sections) else ''
        print(f'\n【{level}】{title}')
        print(f'{content}...')
        count += 1
        if count >= 3:
            break

In [17]:
df = session.sql('SELECT * FROM DOC_CHUNKS ORDER BY CHUNK_INDEX').to_pandas()
print(f'=== チャンク統計 ===')
print(f'総チャンク数: {len(df)}')
print(f'平均チャンク長: {df["CHUNK_LENGTH"].mean():.0f}文字')
print(f'最小チャンク長: {df["CHUNK_LENGTH"].min()}文字')
print(f'最大チャンク長: {df["CHUNK_LENGTH"].max()}文字')
print(f'\n=== チャンクサンプル（最初の3件）===')
for idx, row in df.head(3).iterrows():
    headers = row['CHUNK_HEADERS']
    if isinstance(headers, str):
        headers = json.loads(headers)
    print(f"\n【チャンク {row['CHUNK_INDEX']}】")
    print(f"ヘッダー: {headers}")
    print(f"テキスト: {row['CHUNK_TEXT'][:200]}...")

---
## まとめ

### 使用したCortex AI関数

| 関数 | 用途 | 出力 |
|------|------|------|
| `AI_PARSE_DOCUMENT` | ドキュメント解析・画像抽出 | Markdown + 画像データ |
| `AI_EXTRACT` | キーワード・情報抽出 | 構造化データ |
| `AI_AGG` | テキスト要約 | 要約テキスト |
| `AI_COMPLETE` (Multimodal) | 画像分析 | 画像説明テキスト |
| `SPLIT_TEXT_MARKDOWN_HEADER` | チャンク分割 | チャンク配列 |

### 分析フロー

```
PDFファイル
    ↓ AI_PARSE_DOCUMENT (LAYOUT + extract_images)
Markdownテキスト + 画像
    ├─→ AI_EXTRACT → 機能名・キーワード
    ├─→ AI_AGG → 要約
    ├─→ SPLIT_TEXT_MARKDOWN_HEADER → チャンク
    └─→ ステージ保存 → AI_COMPLETE (Multimodal) → 画像分析
```

### ポイント

1. **画像抽出**: `extract_images: true`オプションで埋め込み画像をBase64で取得
2. **ステージ保存**: Pythonプロシージャで画像をデコードしステージに保存
3. **マルチモーダル分析**: `pixtral-large`モデルでステージ上の画像を直接分析
4. **チャンク分割**: Markdownヘッダー構造を活用した意味のあるチャンク生成
5. **RAG対応**: チャンクにヘッダー情報を付与することで検索精度向上

In [17]:
print('=' * 60)
print('ドキュメント分析完了')
print('=' * 60)
df_parsed = session.sql('SELECT PARSED_CONTENT FROM DOC_PARSED').to_pandas()
parsed = df_parsed['PARSED_CONTENT'].iloc[0]
if isinstance(parsed, str):
    parsed = json.loads(parsed)
df_chunks = session.sql('SELECT COUNT(*) AS cnt FROM DOC_CHUNKS').to_pandas()
df_images = session.sql("SELECT COUNT(*) AS cnt FROM DIRECTORY('@ai_discover.public.images')").to_pandas()
print(f"\nドキュメント: snowflake_key_concept.pdf")
print(f"ページ数: {parsed.get('metadata', {}).get('pageCount', 0)}")
print(f"抽出画像数: {df_images['CNT'].iloc[0]}")
print(f"生成チャンク数: {df_chunks['CNT'].iloc[0]}")
print(f"\n生成テーブル:")
print("  - DOC_PARSED (解析結果)")
print("  - DOC_EXTRACTED (抽出情報)")
print("  - DOC_SUMMARY (要約)")
print("  - DOC_IMAGE_ANALYSIS (画像分析)")
print("  - DOC_CHUNKS (チャンク)")
print(f"\n生成ステージ:")
print("  - @ai_discover.public.images (抽出画像)")